In [63]:
import numpy as np
from numpy import linalg as LA
import math
'''
  Funkcija koja vrsi noriranje datog vektora
'''

def vector_norm(p):
  sum = 0
  for x in p:
    sum += x**2
  return math.sqrt(sum)

def normalize(p):
  norm_ = vector_norm(p)
  return p/norm_
'''
  Funkcija prima matricu A koja je ortoganalna i vazi det(A) = 1
  i vraca jedinicni vektor p i ugao fi iz [0, pi] takav da vazi
  da je matrica A matrica rotacije oko prave p za ugao fi
'''
def AxisAngle(A):
  if round(LA.det(A)) != 1:
    print("Determinanta matrice A je razlicita od 1!")
    return
  '''
  TODO
  if A @ A.T == np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]):
    print("Matrica nije ortogonalna!")
    return
  '''
  
  B = A - np.eye(3)

  first = B[0]
  second = B[1]
  third = B[2]

  p = np.cross(first, second)
  if not np.any(p):
    p = np.cross(first, third)
    if not np.any(p):
      p = np.cross(second, third)
  p = p/LA.norm(p) # Trazeni jedinicni vektor p

  u = first # Vektor u je normalan na vektor p
  #print(u) 
  if not np.any(u):
    u = second
    if not np.any(u):
      u = third
  u = u/LA.norm(u)
  #print(u)
  
  u_bar = A @ u
  u_bar = normalize(u_bar)
  phi = np.arccos(u @ u_bar)

  mesoviti = LA.det(np.array([u, u_bar, p]))
  if mesoviti < 0:
    p = -p

  return (p, phi)

p, phi = AxisAngle(np.array([[1/9, -8/9, -4/9], [4/9, 4/9, -7/9], [8/9, -1/9, 4/9]]))
#p, phi = AxisAngle(np.array([[1/9, 8/9, -4/9], [-4/9, 4/9, 7/9], [8/9, 1/9, 4/9]]))

print(f"Jedinicni vektor oko kog se rotira: {p}")
print(f"Ugao za koji se rotira: {phi}")

Jedinicni vektor oko kog se rotira: [ 0.33333333 -0.66666667  0.66666667]
Ugao za koji se rotira: 1.5707963267948966


In [64]:
def izmnozi(p):
  p_t = np.array([[p[0]], [p[1]], [p[2]]])
  return np.array([p_t[0]*p, p_t[1]*p, p_t[2]*p])
'''
  Funkcija koja racuna matricu rotacije oko prave p za ugao fi
  Kao ulaz prima vektor kolonu oko kog se vrsi rotacija i ugao rotacije
'''

def Rodrigez(p, phi):
  p = normalize(p)
  p_pt = izmnozi(p)
  p_x = np.array([[0, -p[2], p[1]], [p[2], 0, -p[0]], [-p[1], p[0], 0]])

  return p_pt + math.cos(phi)*(np.eye(3) - p_pt) + math.sin(phi)*p_x

p, phi = AxisAngle(np.array([[1/9, 8/9, -4/9], [-4/9, 4/9, 7/9], [8/9, 1/9, 4/9]]))

#A = Rodrigez(np.array([1, 0, 0]), 0)
A = Rodrigez(np.array([0.25056281, 0.93511313, 0.25056281]),  1.3696838321801164)
print("Matrica rotacije dobijena Rodrigezovom formulom: ")
print()
print(A)

Matrica rotacije dobijena Rodrigezovom formulom: 

[[ 0.25       -0.0580127   0.96650635]
 [ 0.4330127   0.89951905 -0.0580127 ]
 [-0.8660254   0.4330127   0.25      ]]


In [65]:
'''
  Funkcija prima ortogonalnu matricu A i vazi det(A) = 1
  i vraca Ojlerove uglove phi, tetha, psi redom
'''
def A2Euler(A):
  if round(LA.det(A)) != 1:
    print("Determinanta matrice A je razlicita od 1!")
    return
  '''
  TODO
  if A @ A.T == np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]):
    print("Matrica nije ortogonalna!")
    return
  '''

  if (A[2][0] < 1 and A[2][0] > -1):
    psi = math.atan2(A[1][0], A[0][0])
    tetha = math.asin(-A[2][0])
    phi = math.atan2(A[2][1], A[2][2])
    return phi, tetha, psi

  if (A[2][0] == -1):
    phi = 0
    tetha = math.pi/2
    psi = math.atan2(-A[0][1], A[1][1])
    return phi, tetha, psi

  phi = 0
  tetha = -math.pi/2
  psi = math.atan2(-A[0][1], A[1][1])
  return phi, tetha, psi

phi, tetha, psi = A2Euler(np.array([[1/9, -8/9, -4/9], [4/9, 4/9, -7/9], [8/9, -1/9, 4/9]]))
print("Trazeni Ojlerovi uglovi:")
print(f"Psi: {psi}")
print(f"Tetha: {tetha}")
print(f"Phi: {phi}")


Trazeni Ojlerovi uglovi:
Psi: 1.3258176636680326
Tetha: -1.09491407713448
Phi: -0.24497866312686414


In [66]:
def Euler2A(phi, tetha, psi):
  Rx_phi = Rodrigez(np.array([1, 0, 0]), phi)
  Ry_tetha = Rodrigez(np.array([0, 1, 0]), tetha)
  Rz_psi = Rodrigez(np.array([0, 0, 1]), psi)

  return np.matmul(np.matmul(Rz_psi, Ry_tetha), Rx_phi)
  #return Rz_psi @ Ry_tetha @ Rx_phi

A = Euler2A(np.pi/3, np.pi/3, np.pi/3)
print("Trazena matrica rotacije:")
print()
print(A)

Trazena matrica rotacije:

[[ 0.25       -0.0580127   0.96650635]
 [ 0.4330127   0.89951905 -0.0580127 ]
 [-0.8660254   0.4330127   0.25      ]]


In [67]:
def AngleAxis2Q(p, phi):
  w = math.cos(phi/2)
  p = normalize(p)

  x = math.sin(phi/2) * p[0]
  y = math.sin(phi/2) * p[1]
  z = math.sin(phi/2) * p[2]

  return x, y, z, w

# TODO
# Kod mene nema nekog minusa kao kod profesora, istestiraj jos malo
x, y, z, w = AngleAxis2Q(np.array([1/3, -2/3, 2/3]), math.pi/2)

print(f"Trazeni kvaternion je: {x}*i {f'- {-y}' if y < 0 else f'+ {y}'}*j {f'- {-z}' if z < 0 else f'+ {z}'}*k {f'- {-w}' if w < 0 else f'+ {w}'}")

Trazeni kvaternion je: 0.2357022603955158*i - 0.4714045207910316*j + 0.4714045207910316*k + 0.7071067811865476


In [68]:
def Q2AngleAxis(q):
  q = q/LA.norm(q)

  if q[3] < 0:
    q = -q

  phi = 2*math.acos(q[3])

  if abs(q[3]) == 1:
    p = np.array([1, 0, 0])
    return p, phi

  p = normalize(np.array([q[0], q[1], q[2]]))
  return p, phi

p, phi = Q2AngleAxis(np.array([0.2357022603955158, -0.4714045207910316, 0.4714045207910316, 0.7071067811865476]))
print(f"Prava oko koje se rotira je: [{p[0]}, {p[1]}, {p[2]}]")
print(f"Ugao za koji se rotira je: {phi}")

Prava oko koje se rotira je: [0.3333333333333333, -0.6666666666666666, 0.6666666666666666]
Ugao za koji se rotira je: 1.5707963267948966


In [69]:
def test():
  # Profesorov test primer
  # phi = -np.arctan(1/4)
  # theta = -np.arcsin(8/9)
  # psi =  np.arctan(4)

  # Moj test primer
  phi = np.pi/3
  theta = np.pi/3
  psi = np.pi/3

  # Ojlerovi uglovi
  print('Ojlerovi uglovi:')
  print('\u03D5 =', phi)
  print('\u03B8 =', theta)
  print('\u03C8 =', psi)
  print()

  # Matrica rotacije
  print('Euler2A:')
  A = Euler2A(phi, theta, psi)
  print('A =')
  print(A)
  print()

  # Osa i ugao
  print('AxisAngle:')
  p, phi0 = AxisAngle(A)
  print('p =', p)
  print('\u03D5 =', phi0)
  print()

  # Vracanje na matricu
  print('Rodrigez:')
  A = Rodrigez(p, phi0)
  print('A =')
  print(A)
  print()

  # Vracanje na uglove
  print('A2Euler:')
  phi, theta, psi = A2Euler(A)
  print('\u03D5 =', phi)
  print('\u03B8 =', theta)
  print('\u03C8 =', psi)
  print()

  # Kvaternion
  print('AxisAngle2Q:')
  q = AngleAxis2Q(p, phi0)
  x, y, z, w = q
  print(f'q = {w:f} {x:+f}i {y:+f}j {z:+f}k')
  print()

  # Vracanje na osu i ugao
  print('Q2AxisAngle:')
  p, phi = Q2AngleAxis(q)
  print('p =', p)
  print('\u03D5 =', phi)

test()

Ojlerovi uglovi:
ϕ = 1.0471975511965976
θ = 1.0471975511965976
ψ = 1.0471975511965976

Euler2A:
A =
[[ 0.25       -0.0580127   0.96650635]
 [ 0.4330127   0.89951905 -0.0580127 ]
 [-0.8660254   0.4330127   0.25      ]]

AxisAngle:
p = [0.25056281 0.93511313 0.25056281]
ϕ = 1.3696838321801164

Rodrigez:
A =
[[ 0.25       -0.0580127   0.96650635]
 [ 0.4330127   0.89951905 -0.0580127 ]
 [-0.8660254   0.4330127   0.25      ]]

A2Euler:
ϕ = 1.0471975511965974
θ = 1.0471975511965976
ψ = 1.0471975511965974

AxisAngle2Q:
q = 0.774519 +0.158494i +0.591506j +0.158494k

Q2AxisAngle:
p = [0.25056281 0.93511313 0.25056281]
ϕ = 1.3696838321801161
